In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import gc
gc.enable()

In [3]:
PATH = '../finanz_vs_news/'

In [4]:
df_reu = pd.read_csv(PATH+'reuters_2008.csv.gz', parse_dates=['Time']).set_index('Time')

In [5]:
df_reu.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 771666 entries, 2008-01-02 04:59:53 to 2008-08-20 04:00:00
Data columns (total 1 columns):
Short_Msg    771666 non-null object
dtypes: object(1)
memory usage: 11.8+ MB


In [6]:
#temporary measure to reduce suspicious dates
df_reu = df_reu.loc['2008',:]

### Read in company ticker names

In [7]:
companies = pd.read_pickle('TickerDic.pickle')

In [8]:
companies.head()

,Ticker,Name,Category Name,RedName
0,AFL,Aflac Incorporated,Accident & Health Insurance,Aflac
1,AIZ,"Assurant, Inc.",Accident & Health Insurance,Assurant
23,UNM,Unum Group,Accident & Health Insurance,Unum
52,IPG,"The Interpublic Group of Companies, Inc.",Advertising Agencies,Interpublic
57,OMC,Omnicom Group Inc.,Advertising Agencies,Omnicom


In [9]:
company_names = companies['RedName'].tolist()

In [10]:
companies.query('RedName == "Morgan Stanley"')

,Ticker,Name,Category Name,RedName
12065,MS,Morgan Stanley,Investment Brokerage - National,Morgan Stanley


### Red in neg/pos dictionaries

In [11]:
pos_words = pd.read_pickle('DicPositiveWords.pickle')
neg_words = pd.read_pickle('DicNegativeWords.pickle')

### Do bag of words

In [23]:
import helpers

In [24]:
media_attention = helpers.count_sensitive_words(df_reu.copy(deep=True), 
                                        ["Morgan Stanley", "Goldman Sachs", "BlackRock"]+[{'positive': pos_words.values.tolist()},
                                                       {'negative': neg_words.values.tolist()}
                                                      ], 
                                        companies_neg_pos=["Morgan Stanley", "Goldman Sachs", "BlackRock"])

In [33]:
media_attention = helpers.fe(media_attention)

In [35]:
media_attention.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 366 entries, 2008-01-01 to 2008-12-31
Freq: D
Data columns (total 23 columns):
Morgan_Stanley_count             366 non-null int64
Goldman_Sachs_count              366 non-null int64
BlackRock_count                  366 non-null int64
positive_count                   366 non-null int64
negative_count                   366 non-null int64
Morgan_Stanley_positive_count    366 non-null int64
Morgan_Stanley_negative_count    366 non-null int64
Goldman_Sachs_positive_count     366 non-null int64
Goldman_Sachs_negative_count     366 non-null int64
BlackRock_positive_count         366 non-null int64
BlackRock_negative_count         366 non-null int64
total_msg_count                  366 non-null int64
Morgan_Stanley_frac              366 non-null float64
Goldman_Sachs_frac               366 non-null float64
BlackRock_frac                   366 non-null float64
positive_frac                    366 non-null float64
negative_frac               

In [27]:
media_attention.sum().sort_values(ascending=False)

total_msg_count                  771472
negative_count                   266333
positive_count                   125720
Morgan_Stanley_count               3726
Morgan_Stanley_negative_count      2453
Goldman_Sachs_count                2302
Goldman_Sachs_negative_count       1902
BlackRock_negative_count           1291
BlackRock_count                     939
Morgan_Stanley_positive_count       155
BlackRock_positive_count             48
Goldman_Sachs_positive_count         28
dtype: int64

In [38]:
print('Numbers of negative and positive mentions of Morgan Stanley are: {} and {}'.format(media_attention['Morgan_Stanley_negative_count'].sum(),
                                                                                   media_attention['Morgan_Stanley_positive_count'].sum())
     )

Numbers of negative and positive mentions of Morgan Stanley are: 2453 and 155


#### Save the pre-processed media dataset

In [ ]:
media_attention.to_csv('MediaAttention_Mini.csv')